In [56]:
from ThesisPackage.Environments.multi_pong_sender_receiver_ball_onehot import PongEnvSenderReceiverBallOneHot
from ThesisPackage.RL.Centralized_PPO.multi_ppo import PPO_Multi_Agent_Centralized
from ThesisPackage.RL.Decentralized_PPO.util import flatten_list, reverse_flatten_list_with_agent_list
from ThesisPackage.Wrappers.vecWrapper import PettingZooVectorizationParallelWrapper
import torch
import numpy as np
import time
import os
import matplotlib.pyplot as plt
import torch.nn.functional as F
import copy

In [57]:
def make_env(max_episode_steps = 1024, sequence_length = 1, vocab_size = 3):
    env = PongEnvSenderReceiverBallOneHot(width=20, height=20, vocab_size=vocab_size, sequence_length=sequence_length, max_episode_steps=max_episode_steps)
    # env = ParallelFrameStack(env, 4)
    return env

In [58]:
def load(path="models/checkpoints", sequence_length=1, vocab_size=3):
    env = make_env(sequence_length=sequence_length, vocab_size=vocab_size)
    models = {}
    for model in os.listdir(path):
        if "pong" in model:
            state_dict = torch.load(os.path.join(path, model))
            timestamp = model.split("_")[-1]
            timestamp = int(timestamp.split(".")[0])
            agent = PPO_Multi_Agent_Centralized(env, device="cpu")
            agent.agent.load_state_dict(state_dict)
            models[timestamp] = agent
    return models

In [59]:
def perturbation(inputs, model, vocab_size, sequence_length):
    
    # Extract environment inputs
    environment_inputs = inputs[:, :-1 * vocab_size * sequence_length]

    # Extract original logits
    inputs = torch.tensor(inputs, dtype=torch.float32)
    original_logits = model(inputs)
    original_logits = F.softmax(original_logits, dim=1).detach().numpy()
    original_logits = F.log_softmax(torch.tensor(original_logits), dim=1).detach()

    perturbation_logits = []
    for token in range(vocab_size):
        # One-hot encoded sequence of tokens
        utterances = np.array([token for _ in range(sequence_length)])
        utterances = np.eye(vocab_size)[utterances].flatten()
        utterances = np.expand_dims(utterances, axis=0)
        utterances = np.repeat(utterances, inputs.shape[0], axis=0)

        # Concatenate environment inputs with utterances
        perturbation_inputs = np.concatenate((environment_inputs, utterances), axis=1)
        perturbation_inputs = torch.tensor(perturbation_inputs, dtype=torch.float32)

        # Get logits for perturbed inputs
        current_logits = model(perturbation_inputs).detach().numpy()
        current_logits = F.softmax(torch.tensor(current_logits), dim=1).detach().numpy()

        perturbation_logits.append(current_logits)

    divergences = []
    for input_array in perturbation_logits:
        kl_divergences = []
        for i in range(len(input_array)):
            q = F.softmax(torch.tensor(input_array[i]), dim=0)
            kl_div = F.kl_div(original_logits, q, reduction='batchmean').item()
            kl_divergences.append(kl_div)

        divergences.append(kl_divergences)
    max_divergences = np.max(divergences, axis=0)
    return max_divergences

In [60]:
def test_perturbation(env, agent, threshold=0.02, number_samples=30000):
    language_importances = []
    obs, info = env.reset()
    state = env.state()
    average_length = []
    tokens = []
    data = {"paddle_1": [], "paddle_2": [], "paddle_1_obs": [], "paddle_2_obs": [], "ball_1_pos": [], "ball_2_pos": []}

    timestep = 0
    while True:
        for cur_agent in env.agents:
            data[cur_agent].append(copy.deepcopy(env.paddles[cur_agent]))
            data[cur_agent + "_obs"].append(copy.deepcopy(obs[cur_agent]))
        for cur_ball in env.balls.keys():
            data[cur_ball + "_pos"].append(copy.deepcopy(env.balls[cur_ball]["position"]))
        obs = [obs]
        state = [state]
        obs = np.array(flatten_list(obs))
        state = np.array(flatten_list(state))
        
        # integrated_grads = smoothgrad(obs_track, agent.agent.actor, 0, sigma=1.0, steps=30)
        language_perturbation = perturbation(obs, agent.agent.actor, env.vocab_size, env.sequence_length)
        language_importances.append(language_perturbation)

        if np.any(language_perturbation > threshold):
            timestep += 1

        if timestep > number_samples:
            break

        obs = torch.tensor(obs, dtype=torch.float32)
        state = torch.tensor(state, dtype=torch.float32)
        with torch.no_grad():
            actions, _, _, _ = agent.agent.get_action_and_value(obs, state)
            actions = reverse_flatten_list_with_agent_list(actions, agent.agents)

        actions = actions[0]
        actions = {agent: action.cpu().numpy() for agent, action in actions.items()}

        obs, _, truncations, terminations, infos = env.step(actions)
        state = env.state()

        if any([truncations[agent] or terminations[agent] for agent in env.agents]):
            average_length.append(env.timestep)
            obs, info = env.reset()
            state = env.state()
    return np.array(language_importances), data, average_length

In [61]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Step 2: Define the dataset and dataloader
class PositionDataset(Dataset):
    def __init__(self, data, labels, device):
        self.data = torch.tensor(data, dtype=torch.float32, device=device)
        self.labels = torch.tensor(labels, dtype=torch.long, device=device)
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# Step 3: Define the model architecture
class SimpleClassifier(nn.Module):
    def __init__(self, input_size):
        super(SimpleClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 2)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [62]:
def train(num_epochs, dataloader, input_size, learning_rate, device):
    
    model = SimpleClassifier(input_size).to(device)

    # Step 4: Train the model
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=80, gamma=0.5)

    for epoch in range(num_epochs):
        running_loss = 0.0
        correct_predictions = 0
        total_predictions = 0
        
        for inputs, targets in dataloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            
            # Accumulate loss
            running_loss += loss.item()
            
            # Calculate accuracy
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == targets).sum().item()
            total_predictions += targets.size(0)
        
        epoch_loss = running_loss / len(dataloader)
        epoch_accuracy = correct_predictions / total_predictions

        # scheduler.step()
        
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}")

    print("Training complete!")
    return epoch_accuracy, model

In [63]:
def test(model, dataloader, device):
    correct_predictions = 0
    total_predictions = 0
    with torch.no_grad():
        for inputs, targets in dataloader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == targets).sum().item()
            total_predictions += targets.size(0)
    
    accuracy = correct_predictions / total_predictions
    print(f"Accuracy: {accuracy:.4f}")
    return accuracy

In [64]:
results = {}

In [65]:
def calculate_chance_level(labels):
    unique, counts = np.unique(labels, return_counts=True)
    index = np.argmax(counts)

    chance_level = counts[index] / len(labels)
    return chance_level

In [66]:
import math
from tqdm import tqdm


def generate_training_data(language_importances_indices, data, sequence_length=1, vocab_size=3):
    paddle_1_indices = np.where(language_importances_indices[1] == 0)
    paddle_1_indices = language_importances_indices[0][paddle_1_indices][3:-3]
    
    inputs = []
    labels = []

    labels = np.zeros(len(paddle_1_indices))
    labels = np.where(np.array(data["paddle_1"])[paddle_1_indices] > np.array(data["paddle_2"])[paddle_1_indices], 1, 0)

    paddle_1_obs = np.array(data["paddle_1_obs"])
    paddle_1_obs = np.array([paddle_1_obs[index - 3:index + 3] for index in paddle_1_indices])
    player_1_lang = paddle_1_obs[:, :, -1 * sequence_length * vocab_size:]
    shape = player_1_lang.shape
    new_shape = (shape[0], shape[1] * shape[2])
    player_1_lang = player_1_lang.reshape(new_shape)

    paddle_2_obs = np.array(data["paddle_2_obs"])
    paddle_2_obs = np.array([paddle_2_obs[index - 3:index + 3] for index in paddle_1_indices])
    player_2_lang = paddle_2_obs[:, :, -1 * sequence_length * vocab_size:]
    shape = player_2_lang.shape
    new_shape = (shape[0], shape[1] * shape[2])
    player_2_lang = player_2_lang.reshape(new_shape)

    inputs = np.concatenate((player_1_lang, player_2_lang), axis=1)

    inputs = np.array(inputs)
    labels = np.array(labels)

    return inputs, labels

def generate_training_data_obs(language_importances_indices, data, sequence_length=1, vocab_size=3):
    paddle_1_indices = np.where(language_importances_indices[1] == 0)
    paddle_1_indices = language_importances_indices[0][paddle_1_indices]

    paddle_1_data = np.array(data["paddle_1_obs"])[paddle_1_indices]
    labels = np.zeros_like(paddle_1_indices)
    labels = np.where(np.array(data["paddle_1"])[paddle_1_indices] > np.array(data["paddle_2"])[paddle_1_indices], 1, 0)

    paddle_1_data_no_lang = paddle_1_data[:, :-1 * sequence_length * vocab_size]

    return paddle_1_data, labels, paddle_1_data_no_lang

In [67]:
import os
directories = os.listdir("/Users/cowolff/Documents/GitHub/ma.pong_rl/Plotting/saliencies_live/Multi_Pong/")
directories = [model for model in directories if ".DS_Store" not in model]
models = {}
for directory in directories:
    sequence_length = int(directory.split("_")[-1])
    agents = load(f"/Users/cowolff/Documents/GitHub/ma.pong_rl/Plotting/saliencies_live/Multi_Pong/{directory}", sequence_length=sequence_length)
    agent_indizes = list(agents.keys())
    agent_indizes.sort()
    models[sequence_length] = agents[agent_indizes[-1]]

/Users/cowolff/miniconda3/envs/thesis/lib/python3.10/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float16
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [68]:
results = {}
for seq, agent in models.items():
    env = make_env(sequence_length=seq)
    results[seq] = {}

    language_importances, data, average_length = test_perturbation(env, agent, number_samples=300000)
    language_importances_larger = np.where(language_importances > 0.02)
    larger_inputs, larger_labels = generate_training_data(language_importances_larger, data, sequence_length=seq, vocab_size=3)
    training_results = {}
    for batch_size in [8, 16, 32]:
        dataset = PositionDataset(larger_inputs, larger_labels, "cpu")
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        input_size = larger_inputs.shape[1]
        accuracy, model = train(120, dataloader, input_size, 0.001, "cpu")

        test_importances, test_data, test_labels = test_perturbation(env, agent, number_samples=10000)
        test_importances_larger = np.where(test_importances > 0.02)
        test_inputs, test_labels = generate_training_data(test_importances_larger, test_data, sequence_length=seq, vocab_size=3)
        test_dataset = PositionDataset(test_inputs, test_labels, "cpu")
        test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)
        test_accuracy = test(model, test_dataloader, "cpu")

        training_results[batch_size] = test_accuracy

    print(training_results)

    results[seq]["above threshold"] = test_accuracy
    

Epoch 1/120, Loss: 0.5356, Accuracy: 0.6868
Epoch 2/120, Loss: 0.5299, Accuracy: 0.6892
Epoch 3/120, Loss: 0.5284, Accuracy: 0.6901
Epoch 4/120, Loss: 0.5278, Accuracy: 0.6911
Epoch 5/120, Loss: 0.5273, Accuracy: 0.6912
Epoch 6/120, Loss: 0.5269, Accuracy: 0.6903
Epoch 7/120, Loss: 0.5267, Accuracy: 0.6914
Epoch 8/120, Loss: 0.5263, Accuracy: 0.6910
Epoch 9/120, Loss: 0.5263, Accuracy: 0.6915
Epoch 10/120, Loss: 0.5262, Accuracy: 0.6905
Epoch 11/120, Loss: 0.5259, Accuracy: 0.6904
Epoch 12/120, Loss: 0.5260, Accuracy: 0.6908
Epoch 13/120, Loss: 0.5259, Accuracy: 0.6907
Epoch 14/120, Loss: 0.5258, Accuracy: 0.6906
Epoch 15/120, Loss: 0.5258, Accuracy: 0.6912
Epoch 16/120, Loss: 0.5258, Accuracy: 0.6914
Epoch 17/120, Loss: 0.5256, Accuracy: 0.6908
Epoch 18/120, Loss: 0.5258, Accuracy: 0.6912
Epoch 19/120, Loss: 0.5255, Accuracy: 0.6911
Epoch 20/120, Loss: 0.5257, Accuracy: 0.6909
Epoch 21/120, Loss: 0.5255, Accuracy: 0.6903
Epoch 22/120, Loss: 0.5255, Accuracy: 0.6914
Epoch 23/120, Loss:

In [69]:
print(results)

{1: {'above threshold': 0.6827969711310933}, 2: {'above threshold': 0.7241631162507608}, 3: {'above threshold': 0.7144892434819775}}


In [70]:
def generate_latex_table(data):
    latex_table = "\\begin{table}[h!]\n\\centering\n\\begin{tabular}{|c|c|c|c|}\n"
    latex_table += "\\hline\n"
    latex_table += "\\textbf{Seq} & \\textbf{All} & \\textbf{\\textless T=0.02} & \\textbf{\\textgreater T=0.02} \\\\\n"
    latex_table += "\\hline\n"

    for seq_len, values in data.items():
        above_threshold_len = values['below threshold']
        noised_share_len = values['above threshold']
        all_utterances = values['all']
        latex_table += f"{seq_len} & {all_utterances:.3f} & {above_threshold_len:.3f} & {noised_share_len:.3f} \\\\\n"
        latex_table += "\\hline\n"

    latex_table += "\\end{tabular}\n\\caption{MA Pong Diagnostic classifier results}\n\\label{table:ma_pong_classifier}\n\\end{table}\n"
    return latex_table

# Generate and print the LaTeX table
latex_table = generate_latex_table(results)
print(latex_table)

KeyError: 'below threshold'